# Pandas Cookbook

In [1]:
import pandas as pd
import json
import re

## DataFrame 基礎

### 定義 DataFrame

DataFrame 的定義有幾種不同的方式：
1. dictionary of lists
2. list of lists
3. list of dictionaries

1. dictionary of lists

In [4]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]})
df

,A,B
0,1,11
1,2,12
2,3,13
3,4,14


2. list of lists

In [5]:
L=[
    [1,11], 
    [2, 12],
    [3, 13], 
    [4, 14]]
df = pd.DataFrame(L, columns=['A','B'])
df

,A,B
0,1,11
1,2,12
2,3,13
3,4,14


3. list of dictionaries

In [6]:
L=[{'A':1, 'B':11}, 
   {'A':2, 'B':12},
   {'A':3, 'B':13}, 
   {'A':4, 'B':14}]
df = pd.DataFrame(L)
df

,A,B
0,1,11
1,2,12
2,3,13
3,4,14


### 新增加資料列
新增資料列有幾種不同的方式：
1. loc append
2. DataFrame append
3. 轉成 list of recoreds

In [15]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]})
df

,A,B
0,1,11
1,2,12
2,3,13
3,4,14


1. loc

loc 將索引當作類似 dict 的 key， 如果 key 不存在，df 就會安排新的 row 放在 DataFrame 最後面，而且以索引值為新 key。

In [16]:
df.loc[4] = [5,15]
df

,A,B
0,1,11
1,2,12
2,3,13
3,4,14
4,5,15


In [17]:
df.loc[5] = {'A':6,'B':16}
df

,A,B
0,1,11
1,2,12
2,3,13
3,4,14
4,5,15
5,6,16


In [18]:
df.loc[6, 'A'] = 7 
df

,A,B
0,1.0,11.0
1,2.0,12.0
2,3.0,13.0
3,4.0,14.0
4,5.0,15.0
5,6.0,16.0
6,7.0,NaN


索引值不一定要按順序，如下例 row -1。

In [19]:
df.loc[-1] = [6,16]
df

,A,B
0,1.0,11.0
1,2.0,12.0
2,3.0,13.0
3,4.0,14.0
4,5.0,15.0
5,6.0,16.0
6,7.0,NaN
-1,6.0,16.0


其次，若 key 已經存在，則該 row 就會被覆蓋(如下例 row 3)。

In [20]:
df.loc[3] = [4.1,14.1]
df

,A,B
0,1.0,11.0
1,2.0,12.0
2,3.0,13.0
3,4.1,14.1
4,5.0,15.0
5,6.0,16.0
6,7.0,NaN
-1,6.0,16.0


iloc 類似於 loc ，但是不能使用於增加 row，因為 iloc 多了幾個限制：
1. iloc 的索引必須為數字
2. iloc 只能在已定義的 row 中修改，不能新增 row
3. iloc 不會修改 index 的值


2. DataFrame append

第二種方法是以 DataFrame 所提供的 append member ，將另一個 DataFrame append 在後面。這個方法的缺點是：
1. 要將 append 的 row 先包裝成 DataFrame。
2. 索引值不會接續下去。

In [22]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]})
df2 = pd.DataFrame([[11, 21]],columns=['A','B'])
df = df.append(df2)
df

,A,B
0,1,11
1,2,12
2,3,13
3,4,14
0,11,21


如果不想將 row 包裝成 DataFrame，也可以直接 append list of records。

In [24]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]})
df = df.append([{'A':11,'B':21}])
df

,A,B
0,1,11
1,2,12
2,3,13
3,4,14
0,11,21


3. 轉成 list of records 

這個方法看起來更麻煩，但是如果數量龐大，可能速度會比較快。方法如下：
1. 將 DataFrame 轉成 records
2. 新增一筆 record
3. 將 records 轉成 DataFrame

In [25]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]})
L = df.to_dict('records')
L.append({ 'A': 11, 'B': 21})
df = pd.DataFrame(L)
df

,A,B
0,1,11
1,2,12
2,3,13
3,4,14
4,11,21


4. 轉成 list of lists

這個方法類似於 list of records，步驟如下：
1. 將 DataFrame 轉成 list of lists
2. 新增一筆 list
3. 將 list of lists 轉成 DataFrame

In [34]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]})
L = df.values.tolist()
L.append([11, 21])
df = pd.DataFrame(L, columns=['A','B'])
df

,A,B
0,1,11
1,2,12
2,3,13
3,4,14
4,11,21


### DataFrame 的定址方式

DataFrame 一共有下列幾種定址方式：
1. loc
2. iloc
3. at
4. iat
5. []
6. ix

1. loc

loc 是直接指定 index label 的方式定址。例如：

In [36]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]},index=['W','X','Y','Z'])
df.loc['X']=[0,0]
df

,A,B
W,1,11
X,0,0
Y,3,13
Z,4,14


2. iloc

iloc 的索引值必須為整數，若使用非數字會發生錯誤，例如
df.iloc['X']=[0,0] 會產生 ValueError 的錯誤。

In [40]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]},index=['W','X','Y','Z'])
df.iloc[1]=[0,0]
df

,A,B
W,1,11
X,0,0
Y,3,13
Z,4,14


3. at 

at 可以直接指定一個 cell，例如：

In [47]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]},index=['W','X','Y','Z'])
df.at['X','A'] = 0
df

,A,B
W,1,11
X,0,12
Y,3,13
Z,4,14


4. iat

iat 同樣可以直接指定一個 cell，但必須使用整數，例如：

In [55]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]},index=['W','X','Y','Z'])
df.iat[1,0]=0 
df

,A,B
W,1,11
X,0,12
Y,3,13
Z,4,14


5. []


In [67]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]},index=['W','X','Y','Z'])
display(df)
df[[False,True,False,False]]


,A,B
W,1,11
X,2,12
Y,3,13
Z,4,14


,A,B
X,2,12


In [71]:
display(df['A']%2==0)
df[df['A']%2==0]

W    False
X     True
Y    False
Z     True
Name: A, dtype: bool

,A,B
X,2,12
Z,4,14


In [72]:
df.loc[df['A']%2==0]

,A,B
X,2,12
Z,4,14


## Excel 檔案讀寫

### 讀取 Excel 檔

read_excel 預設只會讀取第一個工作表，所以，如果 excel 檔只有一個工作表時，
或是只需讀取第一個工作表時，不必使用其他參數。例如：：

In [2]:
# 簡單讀入一個檔案
df=pd.read_excel("data/國教院三等七級詞表_20190618.xls")
df.head()

,等級,詞彙
0,第1級,一
1,第1級,一些
2,第1級,一共/共
3,第1級,一定
4,第1級,一月


如果想讀取的工作表不是第一個工作表，必須以 sheet_name 指定工作表的「名稱」，或是「索引號」。例如：

In [7]:
# 只讀進其中1個表單
df=pd.read_excel("data/國教院三等七級詞表_20190618.xls",sheet_name=3)
df.head() 

,等級,詞彙
0,第4級,一再
1,第4級,一同
2,第4級,一向
3,第4級,一帶
4,第4級,一方面


read_excel 函數也可以一次讀入多個表單，方法是將 sheet_name 參數值設定為 list。但必須注意的是，此時載入的表格不是 DataFrame ，而是 collections 模組的 OrderedDict。其中的每一個 item 儲存一個 DataFrame。
例如：

In [15]:
# 一次讀進7個表單
df=pd.read_excel("data/國教院三等七級詞表_20190618.xls",sheet_name=[0,1,2,3,4,5,6])
type(df) # 注意：多個表單就不是 DataFrame！！
df[0].head()

,等級,詞彙
0,第1級,一
1,第1級,一些
2,第1級,一共/共
3,第1級,一定
4,第1級,一月


In [16]:
df[1].head()

,等級,詞彙
0,第2級,一下子/一下
1,第2級,一半
2,第2級,一塊/一塊兒
3,第2級,一會/一會兒
4,第2級,一直


### 寫入 Excel 檔

寫入 Excel 檔使用 to_excel，寫入單一工作表時，只要直接呼叫 to_excel 即可完成。

In [18]:
df = pd.DataFrame({'Data': [11, 12, 13, 14]})
df.to_excel("data/test1.xlsx")

sheet_name 參數可以指定工作表的名稱：

In [19]:
df.to_excel("data/test1.xlsx",sheet_name='表一')

如果要在一個檔案中寫入多個工作表，必須先寫到 ExcelWriter 物件，再由 ExcelWriter 寫入檔案：

In [20]:
# Create some Pandas dataframes from some data.
df1 = pd.DataFrame({'Data': [11, 12, 13, 14]})
df2 = pd.DataFrame({'Data': [21, 22, 23, 24]})
df3 = pd.DataFrame({'Data': [31, 32, 33, 34]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('data/test2.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df1.to_excel(writer, sheet_name='Sheet1')
df2.to_excel(writer, sheet_name='Sheet2')
df3.to_excel(writer, sheet_name='Sheet3')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

## 從既有欄位產生新欄位

### 以 map 搭配 dict 產生新欄位
這個功能只有 map 做的到， apply 一定要搭配函數。

In [34]:
T={11:'odd',12:'even',13:'odd',14:'even'}
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]})
df['C']=df['B'].map(T)
df

,A,B,C
0,1,11,odd
1,2,12,even
2,3,13,odd
3,4,14,even


### 以 map 搭配函數產生新欄位 

In [32]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]})
df['C']=df['B'].map(lambda x: x**2)
df

,A,B,C
0,1,11,121
1,2,12,144
2,3,13,169
3,4,14,196


### 以 apply 搭配函數產生新欄位 

In [37]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]})
df['C']=df['B'].apply(lambda x: x**2)
df

,A,B,C
0,1,11,121
1,2,12,144
2,3,13,169
3,4,14,196


### 以兩個欄位產生新欄位
這個功能只有 apply 做的到，因為只有 DataFrame 只有 apply 函數，沒有 map 函數。

In [40]:
df = pd.DataFrame({'A':[1,2,3,4], 'B': [11, 12, 13, 14]})
df["AB"]= df.apply(lambda x: (x['A']*x['B']), axis=1)
df

,A,B,AB
0,1,11,11
1,2,12,24
2,3,13,39
3,4,14,56


# DataFrame 的 groupby 運算

### 計算每一類有多少個

In [23]:
df = pd.DataFrame({'Data': ['A','B','C','D','B','C','D','C','D','D']}) 
df

,Data
0,A
1,B
2,C
3,D
4,B
5,C
6,D
7,C
8,D
9,D


In [24]:
gdf=df.groupby('Data')
gdf.size()

Data
A    1
B    2
C    3
D    4
dtype: int64

## DataFrame 的 Select 運算

### 找出符合條件的運算

In [29]:
# 找出 3 的倍數
df = pd.DataFrame({'Data': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]})
three=df.loc[df['Data']%3==0]
three

,Data
1,12
4,15
7,18


## DataFrame 的縱向操作
縱向操作即是會改變 rows 數目的運算

### 串接 dataframe
使用 concat 可以將 DataFrame 無條件串接在一起。
* axis=0: 縱向串接
* ignore_index=True 讓 index 重排

In [4]:
df1 = pd.DataFrame({'Data': [11, 12, 13]})
df2 = pd.DataFrame({'Data': [21, 22, 23]})
df3 = pd.DataFrame({'Data': [31, 32, 33]})

df=pd.concat([df1, df2, df3],axis=0,ignore_index = True)
df

,Data
0,11
1,12
2,13
3,21
4,22
5,23
6,31
7,32
8,33


迴圈逐筆處理資料
--------------------

In [18]:
# 逐筆取出資料
df = pd.DataFrame({'Name':['A','B','C','D'], 'Value': [11, 12, 13, 14]})
for index, row in df.iterrows():
    print(row['Name'],row['Value'])

A 11
B 12
C 13
D 14


In [ ]:
# 逐筆修改資料
for idx,row in df.iterrows():
    isbn=str(row['ISBN'])
    if re.match('[0-9]+$',isbn) and len(isbn)==10:
        isbn13=pyisbn.convert(isbn)
        print(isbn,'-->',isbn13)
        df.at[idx,'ISBN']=isbn13

如何逐筆增加欄位
---------------

In [16]:
T={"第1級":"PreA","第2級":"A1","第3級":"A2","第4級":"B1","第5級":"B2","第6級":"C1","第7級":"C2"}
L=[]
for index, row in table.iterrows():
    old_level=T[row['等級']]
    L.append(old_level)
table["舊等級"]=L

### 如何修改一個 cell 的值

In [13]:
months=['一月份','二月份','三月份','四月份','五月份','六月份','七月份','八月份','九月份','十月份','十一月份','十二月份']
df=pd.DataFrame(columns=['月份','花費'])  
for i in range(12):
    df.loc[i]=[months[i],0]
df.head()

,月份,花費
0,一月份,0
1,二月份,0
2,三月份,0
3,四月份,0
4,五月份,0


In [15]:
df.at[0,'花費']=5000
df.head()

,月份,花費
0,一月份,5000
1,二月份,0
2,三月份,0
3,四月份,0
4,五月份,0


In [16]:
df.set_index('月份',inplace=True)
df.at['一月份','花費']=5000
df.head()

,花費
月份,
一月份,5000
二月份,0
三月份,0
四月份,0
五月份,0


以 dictionary 增加欄位
---------------------

In [21]:
written_freq=json.load(open("data/遠流語料詞頻.json","r",encoding="utf8"))
table['書面語詞頻']=table['詞彙'].map(written_freq)
#table

## 如何更改欄位名稱

In [6]:
df = pd.DataFrame({'a':[1,2], 'b': [10,20]})
df

,a,b
0,1,10
1,2,20


In [7]:
df.columns = ['x', 'y']
df

,x,y
0,1,10
1,2,20


In [8]:
df = pd.DataFrame({'a':[1,2], 'b': [10,20]})
df

,a,b
0,1,10
1,2,20


In [9]:
# 單一欄位
df = df.rename(columns={'a':'x'})
df

,x,b
0,1,10
1,2,20


以 function 增加欄位
---------------------

如果是使用 lambda 函數，則可以使用 map 或 apply 成員函數呼叫。例如：

In [ ]:
table["詞長"]=table["詞匯"].map(lambda x: len(x))
table["詞長"]=table["詞"].apply(lambda x: len(x))

如果是使用外部函數，而且包含而外的參數，則必須使用 apply 成員函數呼叫。例如：

In [24]:
def get_freq(item,T):
    wordL=re.split('[ \t]*/[ \t]*',item)
    cnt=0
    for word in wordL:
        if word in T:
            #print("%s\t%s"%(word,int(T[word])))
            cnt+=T[word]
            continue
        word1=re.sub('[1-9].*$','',word)
        if word1 in T:
            #print word,T[word1]
            #print("%s\t%s"%(word,int(T[word1])))
            cnt+=T[word1]
            continue
        word2=re.sub('\([一-十四]\).*$','',word)
        if word2 in T:
            cnt+=T[word2]
            continue
        word3=re.sub('[ㄅ-ㄦ].*$','',word)
        if word3 in T:
            cnt+=T[word3]
            continue
        print("WARNING:%s[%s] 找不到詞頻"%(word,word1))

    return cnt

In [27]:
table['書面語詞頻']=table["詞彙"].apply(get_freq,args=(written_freq,))

調換欄位
-------

In [28]:
table

,等級,詞彙,書面語詞頻
0,第1級,一,1024601
1,第1級,一些,41396
2,第1級,一共/共,8038
3,第1級,一定,37474
4,第1級,一月,2670
5,第1級,一樣,53878
6,第1級,一起,22917
7,第1級,一點/一點點/一點兒,27938
8,第1級,七,16331
9,第1級,三,86188


In [30]:
table=table[["詞彙","等級","書面語詞頻"]]
table

,詞彙,等級,書面語詞頻
0,一,第1級,1024601
1,一些,第1級,41396
2,一共/共,第1級,8038
3,一定,第1級,37474
4,一月,第1級,2670
5,一樣,第1級,53878
6,一起,第1級,22917
7,一點/一點點/一點兒,第1級,27938
8,七,第1級,16331
9,三,第1級,86188


合併兩個 DataFrame
-----------------

In [3]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo','x'],
                     'value': [1, 2, 3, 5,7]})
df1

,lkey,value
0,foo,1
1,bar,2
2,baz,3
3,foo,5
4,x,7


In [4]:
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                     'value': [5, 6, 7, 8]})
df2

,rkey,value
0,foo,5
1,bar,6
2,baz,7
3,foo,8


In [5]:
# 預設合併以 inner (joint) 的方式進行
df1.merge(df2, left_on='lkey', right_on='rkey')

,lkey,value_x,rkey,value_y
0,foo,1,foo,5
1,foo,1,foo,8
2,foo,5,foo,5
3,foo,5,foo,8
4,bar,2,bar,6
5,baz,3,baz,7


In [10]:
# 指定　outer (union) 可以將兩邊都列出來
# fillna 可以將 NA 改用其他值替代 0
df1.merge(df2, left_on='lkey', right_on='rkey', how="outer").fillna(0)

,lkey,value_x,rkey,value_y
0,foo,1,foo,5.0
1,foo,1,foo,8.0
2,foo,5,foo,5.0
3,foo,5,foo,8.0
4,bar,2,bar,6.0
5,baz,3,baz,7.0
6,x,7,0,0.0


# 抽取某些 rows

In [13]:
df1

,lkey,value
0,foo,1
1,bar,2
2,baz,3
3,foo,5
4,x,7


In [14]:
df1.loc[df1['value'] > 2]

,lkey,value
2,baz,3
3,foo,5
4,x,7


In [15]:
df1.loc[df1.index % 2==0]

,lkey,value
0,foo,1
2,baz,3
4,x,7


輸出到文字檔
-----------

In [31]:
table.to_csv("data/words.txt",sep='\t', encoding='utf-8')

# 處理 NaN 的資料

## 取代所有 NaN 的資料

In [ ]:
df = df.fillna(0)

## 取代特定欄位的 NaN 資料

In [ ]:
df["詞頻"]=df["詞頻"].fillna(0)

In [ ]:
df[["詞頻","每百萬詞頻"]]=df[["詞頻","每百萬詞頻"]].fillna(0)

## 以其他欄來 fillna

In [ ]:
df['col1']=df['col1'].fillna(df['col2'])

## 找出所有包含 NaN 的列

In [ ]:
test[test.isnull().any(axis=1)]

# 排序

In [ ]:
df=df.sort_values(by='col1', ascending=False)

# 索引管理

### 重建索引

In [ ]:
# drop=True 表示要刪掉舊的索引
df=df.reset_index(drop=True)

### 索引從 1 開始

In [ ]:
df.index += 1

In [ ]:
df.index = np.arange(1, len(df)+1)

### 載入檔案時，避免索引被當成 Unnamed: 0 欄位

如果 DataFrame 寫入 excel 檔時，包含了從1開始的索引列，則讀回 DataFrame 時，會重新產生一個索引列。
而原來的索引列，會被標為 Unnamed: 0 欄位，以下為示範的例子：

In [43]:
df = pd.DataFrame({'Data': [11, 12, 13, 14]})
df.index += 1
df.to_excel("data/test1.xlsx")

In [44]:
df=pd.read_excel("data/test1.xlsx")
df

,Unnamed: 0,Data
0,1,11
1,2,12
2,3,13
3,4,14


要解決這個問題可以在乎叫 read_excel 時，以 index_col 參數指定以現存的索引為索引，例如：

In [45]:
df=pd.read_excel("data/test1.xlsx",index_col=0)
df

,Data
1,11
2,12
3,13
4,14


### 建立非數字索引

dataframe 的 index 除了不必從一開始，也不一定為數字。我們可以使用 set_index 函數重新指定一欄為 index。

以月份花費表為例，首先，我們建立一個空的月份花費表：

In [10]:
months=['一月份','二月份','三月份','四月份','五月份','六月份','七月份','八月份','九月份','十月份','十一月份','十二月份']
df=pd.DataFrame(columns=['月份','花費'])  
for i in range(12):
    df.loc[i]=[months[i],0]
df.head()

,月份,花費
0,一月份,0
1,二月份,0
2,三月份,0
3,四月份,0
4,五月份,0


接著以 set_index 指定「月份」為 index：

In [11]:
df.set_index('月份',inplace=True)
df.head()

,花費
月份,
一月份,0
二月份,0
三月份,0
四月份,0
五月份,0
